# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd, numpy as np
from sqlalchemy.engine import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName1', engine)
df.head()
X = df['message'].values
y = df.iloc[:, 4:].values
category_names = list(df.iloc[:, 4:].columns)

### 2. Write a tokenization function to process your text data

In [3]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- First of all,

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.32)

vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = MultiOutputClassifier(RandomForestClassifier())
# train classifier
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
clf.fit(X_train_tfidf, y_train)

# predict on test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)
y_pred.shape

(8389, 36)

Now, convert this into the pipeline model

This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()) )
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.32)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

from sklearn.metrics import classification_report

colnames = list(df.iloc[:, 4:].columns)
for k in range(len(colnames)):
    print(k, '. ', colnames[k], '. \t acc = ', (y_pred[:, k] == y_test[:,k]).mean())
    print(classification_report(y_test[:,k], y_pred[:,k]))

0 .  related . 	 acc =  0.792227917511
             precision    recall  f1-score   support

          0       0.61      0.34      0.44      1984
          1       0.82      0.93      0.87      6405

avg / total       0.77      0.79      0.77      8389

1 .  request . 	 acc =  0.878889021337
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      6940
          1       0.81      0.39      0.52      1449

avg / total       0.87      0.88      0.86      8389

2 .  offer . 	 acc =  0.994039814042
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      8339
          1       0.00      0.00      0.00        50

avg / total       0.99      0.99      0.99      8389

3 .  aid_related . 	 acc =  0.733937298844
             precision    recall  f1-score   support

          0       0.73      0.88      0.79      4929
          1       0.75      0.53      0.62      3460

avg / total       0.74      0.73    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
from sklearn.model_selection import GridSearchCV

parameters = {
                'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
                'vect__max_df': (0.5, 0.75, 1.0),
                'vect__max_features': (None, 100, 500),
                'tfidf__use_idf': (True, False)
            }

#cv = GridSearchCV(pipeline, param_grid = parameters)
#cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
#y_pred = cv.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
parameters = {
                'vect__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)),
                'vect__max_df': (0.5, 0.75, 1.0),
                'vect__max_features': (None, 500, 1000, 2000),
                'tfidf__use_idf': (True, False)
            }

### 9. Export your model as a pickle file

In [12]:
import pickle
filename = 'classifier.pkl'
pickle.dump(clf, open(filename, 'wb'))
#pickle.dump(clf, filename)

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [14]:
import sys
len(sys.argv), sys.argv

(3,
 ['/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py',
  '-f',
  '/root/.local/share/jupyter/runtime/kernel-b21cc771-6152-4dd6-bfd6-545ad154f00e.json'])